# 8章 MNISTを使った数字認識

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from torchviz import make_dot
from tqdm import tqdm


In [ ]:
relu = nn.ReLU()
x_np = np.arange(-2, 2.1, 0.25)
x    = torch.tensor(x_np).float()
y = relu(x)

plt.plot(x.data, y.data)
plt.title('ReLU関数')
plt.show()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)# cudaが利用可能だと0

## 8.8 データ準備1(データセットによる読み込み)

In [ ]:
data_root = './data'
train_set0 = datasets.MNIST(
    root = data_root,
    train = True,
    download = True
)
!ls -lR ./data/MNIST

In [ ]:
print("データ件数：", len(train_set0))
image, label = train_set0[0]
print("入力データの型：", type(image))
print("正解データの型：", type(label))

In [ ]:
plt.figure(figsize=(1,1))
plt.title(f'{label}')
plt.imshow(image, cmap="gray_r")
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
for i in range(20):
    ax = plt.subplot(2,10,i+1)
    image, label = train_set0[i]
    plt.imshow(image, cmap="gray_r")
    ax.set_title(f'{label}')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 8.9 データ準備2(Transformsによるデータ前処理)

In [ ]:
# ToTensor：入力データの形式をPyTorchで扱えるテンソル形式に変換する
transform1 = transforms.Compose([
    transforms.ToTensor(),
])
train_set1 = datasets.MNIST(
    root=data_root, 
    train=True, 
    download=True,
    transform = transform1
)

In [ ]:
image, label = train_set1[0]
print("入力データの型：", type(image))
print("入力データのshape：", image.shape)
print("Min: ", image.data.min())
print("Max: ", image.data.max())

In [ ]:
# データの範囲を[-1, 1]になるように正規化
# Normalize:X=(x-u)/σ（u=σ=0.5)
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

train_set2 = datasets.MNIST(
    root = data_root, 
    train=True, 
    download=True, 
    transform=transform2
)

image, label = train_set2[0]
print("入力データの型：", type(image))
print("入力データのshape：", image.shape)
print("Min: ", image.data.min())
print("Max: ", image.data.max())

In [ ]:
# Lambdaクラスを利用して１次元化
# テンソルではreshapeではなく、viewを用いる
transform3 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x: x.view(-1)),
])

train_set3 = datasets.MNIST(
    root = data_root, 
    train=True, 
    download=True, 
    transform=transform3
)

image, label = train_set3[0]
print("入力データの型: ", type(image))
print("入力データのshape: ", image.shape)
print("Min: ", image.data.min())
print("Max: ", image.data.max())

In [ ]:
# 最終的なデータセットの定義
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x: x.view(-1)),
])
train_set = datasets.MNIST(
    root = data_root, train=True, download=True, transform=transform
)
test_set = datasets.MNIST(
    root = data_root, train=False, download=True, transform=transform
)

## 8.10 データ準備3(データローダによるミニバッチ用データ生成)

In [ ]:
from torch.utils.data import DataLoader
batch_size=500

train_loader = DataLoader(
    train_set, batch_size = batch_size,
    shuffle = True
)
test_loader = DataLoader(
    test_set, batch_size = batch_size,
    shuffle = False
)

In [ ]:
print(len(train_loader))
for images, labels in train_loader:
    break
print(len(labels))
print(images.shape)
print(labels.shape)

## 8.11 モデル定義

In [ ]:
n_input = image.shape[0]
n_output = len(set(list(labels.data.numpy())))
n_hidden = 128
print(f'n_input: {n_input} n_hidden: {n_hidden} n_output: {n_output}')

In [ ]:
# モデルの定義
class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        return x3


In [ ]:
torch.manual_seed(123)
torch.cuda.manual_seed(123)

net = Net(n_input, n_output, n_hidden)
net = net.to(device)
# net = Net(n_input, n_output, n_hidden).to(device)

In [ ]:
# 学習率
lr = 0.01

# アルゴリズム: 勾配降下法
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

# 損失関数： 交差エントロピー関数
criterion = nn.CrossEntropyLoss()

In [ ]:
for parameter in net.named_parameters():
    print(parameter)

In [ ]:
print(net)

## 8.12 勾配降下法

In [ ]:
for images, labels in train_loader:
    break
inputs = images.to(device)
labels = labels.to(device)
outputs = net(inputs)
print(outputs)

In [ ]:
# 損失計算
loss = criterion(outputs, labels)
print(loss.item())
make_dot(loss, params=dict(net.named_parameters()))

In [ ]:
# 乱数の固定化
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# 学習率
lr = 0.01
# モデル初期化
net = Net(n_input, n_output, n_hidden).to(device)
# 損失関数：交差エントロピー関数
criterion = nn.CrossEntropyLoss()
# 最適化関数
optimizer = optim.SGD(net.parameters(), lr=lr)
# 繰り返し回数
num_epochs = 100
# 評価関数結果記録用
history = np.zeros((0,5))


In [ ]:
# 繰り返し計算メインループ
for epoch in range(num_epochs):
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    # 訓練フェーズ
    for inputs, labels in tqdm(train_loader):
        n_train += len(labels)
        # GPUへ転送
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 勾配の初期化
        optimizer.zero_grad()
        # 予測計算
        outputs = net(inputs)
        # 損失計算
        loss = criterion(outputs, labels)
        # 勾配計算
        loss.backward()
        # パラメータ修正
        optimizer.step()
        # 予測ラベル導出
        predicted = torch.max(outputs, 1)[1]
        # 損失と精度の計算
        train_loss += loss.item()
        train_acc += (predicted == labels).sum().item() 

    # 予測フェーズ
    for inputs_test, labels_test in test_loader:
        n_test += len(labels_test)
        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)

        # 予測計算
        outputs_test = net(inputs_test)

        # 損失関数
        loss_test = criterion(outputs_test, labels_test)

        # 予測データ導出
        predicted_test = torch.max(outputs_test, 1)[1]

        # 損失と制度の計算
        val_loss += loss_test.item()
        val_acc += (predicted_test == labels_test).sum().item()

    # 評価値の算出・記録
    train_acc = train_acc / n_train
    val_acc = val_acc / n_test
    train_loss = train_loss * batch_size / n_train
    val_loss = val_loss * batch_size / n_test
    print (f'Epoch [{epoch+1}/{num_epochs}], loss: {train_loss:.5f} acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}')
    item = np.array([epoch+1 , train_loss, train_acc, val_loss, val_acc])
    history = np.vstack((history, item))
print(f'初期状態：損失：{history[0,3]:.5f}  精度：{history[0,4]:.5f}')
print(f'最終状態：損失：{history[-1,3]:.5f}  精度：{history[-1,4]:.5f}')

In [ ]:
# 学習曲線の表示(損失)
plt.plot(history[:,0], history[:,1], "b", label="train")
plt.plot(history[:,0], history[:,3], "k", label="test")
plt.xlabel('iterate number')
plt.ylabel('loss')
plt.title('learning curve')
plt.legend()
plt.show()

In [ ]:
# 学習曲線の表示(精度)
plt.plot(history[:,0], history[:,2], "b", label="train")
plt.plot(history[:,0], history[:,4], "k", label="test")
plt.xlabel('iterate number')
plt.ylabel('acc')
plt.title('learning curve(acc)')
plt.legend()
plt.show()